In [1]:

import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [ ]:
from langchain_openai import ChatOpenAI

#  보수적인 설정 (일관된, 논리적인 이야기)
llm_before = ChatOpenAI(
    #model="gpt-3.5-turbo-0125",
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.2,  # 낮은 온도로 예측 가능한 출력
    presence_penalty=0.0,  # 기존 패턴 유지
    frequency_penalty=0.0,  # 반복 허용
    max_tokens=150,  # 출력 길이 제한
    top_p=1.0  # 확률 상위 100% 내에서 선택 (제한 없음)
)

#  창의적인 설정 (더 독창적이고 예측 불가능한 이야기)
llm_after = ChatOpenAI(
    #model="gpt-3.5-turbo-0125",
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=1.2,  # 높은 온도로 창의적인 답변 유도
    presence_penalty=1.2,  # 새로운 단어와 개념 유도
    frequency_penalty=0.5,  # 반복을 억제하여 더 다양한 표현 생성
    max_tokens=300,  # 더 긴 이야기 허용
    top_p=0.8  # 제한 없이 다양한 단어 선택 가능
)

# 질문 설정: 짧은 판타지 이야기 생성
# question = "마법의 세계에서 용이 인간과 친구가 되는 짧은 이야기를 써 주세요."
question = "마법의 세계에서 벌어지는 예상치 못한 사건을 주제로 독창적인 짧은 이야기를 만들어 주세요."

# 모델 호출
response_before = llm_before.invoke(question)
response_after = llm_after.invoke(question)

# 결과 출력
print(" Before (논리적이고 보수적인 이야기)")
print(response_before.content)

print("\n-------------------------\n")

print(" After (창의적인 이야기, 더 풍부한 표현)")
print(response_after.content)

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

#  모델 파라미터 (Before: 기본적인 추천 / After: 맞춤형 세부 정보 추가)
before_params = {
    "temperature": 0.7,         # 랜덤성을 적절히 유지 (다양한 답변 유도)
    "max_tokens": 300,          # 응답 길이를 적절히 조절
    "frequency_penalty": 0.5,   # 반복 단어 억제
    "presence_penalty": 0.5,    # 새로운 단어 포함 장려
}

after_params = {
    "temperature": 0.3,         # 창의성을 낮추고 정확한 답변 유도
    "max_tokens": 800,          # 더 긴 답변을 생성 (세부 정보 포함)
    "top_p": 0.85,              # 확률 기반 샘플링 (일관성과 다양성 균형)
    "frequency_penalty": 0.2,   # 반복 단어 감소 (자연스러운 답변)
    "presence_penalty": 0.3,    # 새로운 정보 포함 장려
}

#  두 개의 모델 생성 (Before / After)
#before_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **before_params)
before_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    **before_params
)
#after_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **after_params)
after_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    **after_params
)

#  프롬프트 구성 (올바른 ChatMessagePromptTemplate 사용)
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 여행 전문가입니다. 사용자의 요청에 맞는 최적의 여행지를 추천해 주세요."
)

user_message = HumanMessagePromptTemplate.from_template("{user_input}")

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

#  체인 생성 (Before / After)
before_chain = chat_prompt | before_model
after_chain = chat_prompt | after_model

#  질문 설정 (Before / After의 차이점을 비교할 수 있도록 변경)
test_question = "가족과 함께 3박 4일 동안 한국에서 여유롭게 여행할 수 있는 일정을 동선을 고려하여 자세하게 추천해 주세요."

#  Before 모델 실행
before_response = before_chain.invoke({"user_input": test_question})

#  After 모델 실행
after_response = after_chain.invoke({"user_input": test_question})

#  결과 출력
print(" [Before] 모델 결과")
print(before_response.content)
print("\n" + "="*80 + "\n")  # 가독성을 위한 구분선
print(" [After] 모델 결과")
print(after_response.content)

 [Before] 모델 결과
한국에서 가족과 함께 3박 4일 동안 여유롭게 여행할 수 있는 일정을 추천해 드리겠습니다. 이번 여행은 서울과 가까운 경기도 지역의 명소를 중심으로 계획하였습니다.

**여행일정: 3박 4일**

**1일차: 서울**

*   **아침**: 여행의 시작을 서울의 대표적인 아침 식사 장소인 광화문에서 시작해 보세요. 광화문 근처에는 다양한 한식당이 있습니다. 
*   **오전**: 경복궁을 방문하세요. 경복궁은 한국의 대표적인 궁궐 중 하나로, 아름다운 건축물과 역사적인 의미를 가지고 있습니다. 
*   **오후**: 경복궁 근처의 국립민속박물관을 방문하세요. 이곳에서는 한국의 전통문화와 역사를 배울 수 있습니다.
*   **저녁**: 서울의 대표적인 저녁 식사 장소인 종로에서 저녁을 드세요. 종로에는 다양한 한식당과 중식당이 있습니다.

**2일차: 수원**

*   **아침**: 수원으로 이동하여 수원의 대표 관광지인 수원 화성(수원 성)을 방문하세요. 수원 화성은 조선시대 왕족이 살았던 곳으로, 아름다운 건축물과 역사적인 의미를 가지고 있습니다. 
*   **오후**: 수원 화성의 근처에 위치한 수원시립미술관을 방문하세요. 이곳에서는 다양한 전시회와 문화행사를 즐길 수 있습니다.
*   **저녁**: 수원시의 대표


 [After] 모델 결과
**여행지: 서울**

**일정:**

*   1일차: 서울에 도착하여 호텔에 체크인 후, 점심으로 광화문 근처에 있는 한식당을 방문합니다. 오후에는 경복궁을 방문하여 한국의 역사와 문화를 체험합니다. 저녁에는 광화문 근처에서 저녁 식사를 즐깁니다.
*   2일차: 아침 식사 후, 서울의 대표적인 관광지인 N서울타워를 방문합니다. 타워에서 서울의 아름다운 경치를 감상한 후, 점심으로 타워 근처에 있는 카페에서 식사를 합니다. 오후에는 롯데월드몰을 방문하여 쇼핑과 엔터테인먼트를 즐깁니다. 저녁에는 서울의 대표적인 거리 음식인 타코야키를 맛봅니다.
*   3일차: 아침 식사 후, 서울의 역사

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

#  프롬프트 설정 (천문학 질문에 대한 답변을 생성하는 시스템)
prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 대해 명확하고 자세한 답변을 제공할 수 있습니다."),
    ("user", "{user_input}"),
])

#  기본 모델 설정 (기본적인 답변 생성)
#base_model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=150)  # 150 토큰 제한
base_model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    max_tokens=150
)

#  질문 설정
# 1. MAX_TOKENS 차이를 보여주는 질문 (길이 제한 효과)
max_tokens_question = "인공지능의 발전이 미래 사회에 미칠 영향을 긍정적 측면과 부정적 측면으로 나누어 자세히 설명해 주세요."

# 2. STOP 파라미터 차이를 보여주는 질문 (중단점 효과)
stop_question = "Python 프로그래밍을 배우는 초보자에게 추천하는 학습 단계를 순서대로 설명해 주세요. 각 단계별로 구체적인 방법과 팁을 포함해서 답변해 주세요."

# 3. TEMPERATURE 차이를 보여주는 질문 (창의성 vs 정확성)
temperature_question = "시간 여행이 가능하다면 과거의 어느 시대로 가고 싶은지와 그 이유를 창의적으로 설명해 주세요."

# 4. 복합적 비교를 위한 질문 (모든 파라미터 효과)
complex_question = "화성에 인류가 정착하기 위해 필요한 기술과 준비사항들을 단계별로 설명하고, 각 단계에서 예상되는 도전과제와 해결방안을 제시해 주세요."


question = stop_question

#  Before (기본 max_tokens=150)
messages = prompt.format_messages(user_input=question)
before_answer = base_model.invoke(messages)

#  Before 출력
print("\n [Before] 기본 max_tokens=150 (기본 답변)")
print(before_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

#  After (파라미터 조정 후 비교)
stop_chain = prompt | base_model.bind(max_tokens=150, stop=["."])  # 첫 번째 마침표에서 중단
temp_chain = prompt | base_model.bind(max_tokens=150, temperature=0.8)  # 창의적인 답변 유도

stop_answer = stop_chain.invoke({"user_input": question})
temp_answer = temp_chain.invoke({"user_input": question})

#  After 출력 (stop vs temperature 비교)
print(" [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)")
print(stop_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

print(" [After] max_tokens=150, temperature=0.8 (창의적인 답변)")
print(temp_answer.content)


 [Before] 기본 max_tokens=150 (기본 답변)
Python을 배우고 싶다면 아래 단계를 따라 시작하시길 추천합니다. 

1.  **기본 문법 학습**

    *   Python 공식 문서 (<https://docs.python.org/ko/3/tutorial/index.html> ) 또는 온라인 강의, 튜토리얼을 통해 Python의 기본 문법을 학습합니다. 
    *   변수, 데이터 타입, 제어 구조(조건문, 루프), 함수, 모듈 등에 대해 이해해야 합니다.
    *   기본적인 프로그래밍 개념과 Python의 특성을 익히는 것이 중요합니다.

2.  **실습 및 코딩 연습**

    *   학습한 내용을 바탕으로 다양한 문제를 풀고 코딩 연습을 합니다.
    *   온라인 플랫폼인 LeetCode, HackerRank,


 [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)
1


 [After] max_tokens=150, temperature=0.8 (창의적인 답변)
Python을 처음 배울 때는 단계별로 체계적으로 학습하는 것이 중요합니다. 단계별로 학습하는 방법과 팁을 자세히 알려드리겠습니다.

### 1. Python의 기본 이해

*   **Python 소개**: Python이 무엇인지, 다른 프로그래밍 언어와 어떻게 다른지 이해합니다.
*   **설치**: Python을 다운로드하고 설치합니다. 공식 웹사이트([https://www.python.org/](https://www.python.org/))에서 최신 버전을 설치하는 것을 추천합니다.
*   **IDE 설정**: Python을 작성하고 실행할 수 있는 Integrated Development Environment (IDE)나 텍스트 에디터를 설정합니다. 인기 있는 선택으로는 PyCharm, Visual Studio Code, Jupyter Notebook 등이 있습니다.

### 2. 기초 문
